# DMAS Analysis


## Macros


In [145]:
# MEMORY_BACKEND = "mem0"
MEMORY_BACKEND = "graphiti"

RESET_MEMORY_BACKENDS = True

TOXIC_LATENCY = 0
TOXIC_BANDWIDTH = 0
TOXIC_SLOW_CLOSE = 0
TOXIC_TIMEOUT = 0
TOXIC_SLICER = 0
TOXIC_LIMIT_DATA = 0
TOXIC_RESET_PEER = 0

## Setup and Build


### System Stop


In [98]:
!make -C ../ down

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Stopping everything on ...
cd proxy && docker-compose --env-file ../.env down || echo ""
[+] Running 0/0
 ⠋ Container openai-proxy  Stopping                                        0.1s 
 ⠋ Container ollama-proxy  Stopping                                        0.1s 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[+] Running 0/2
 ⠙ Container openai-proxy  Stopping                                        0.2s 
 ⠙ Container ollama-proxy  Stopping                                        0.2s 
[+] Running 0/2
 ⠹ Container openai-proxy  Stopping                                        0.3s 
 ⠹ Container ollama-proxy  Stopping                                        0.3s 
[+] Running 1/2
 ⠸ Container openai-proxy  Stopping                                        0.4s 
 ✔ Container ollama-proxy  Removed                                         0.3s 
[+] Running 2/2
 ✔ Container openai-proxy  Removed                                         0.5s 
 ✔ Container ollama-proxy  Removed                                         0.3s 
cd dmas && docker-compose --env-file ../.env down || echo ""
[+] Running 0/0
 ⠋ Container coordinator  Stopping                                         0.1s 
[+] Running 0/1
 ⠙ Container coordinator  Stopping                                         0.2s 
[+] Running 0/1
 ⠹ Container coor

### Reset Memory - ( _Optional_ )


In [72]:
if RESET_MEMORY_BACKENDS:
    import subprocess
    subprocess.run(["make", "-C", "../", "reset"], check=True)

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Resetting everything (keeping ollama volume)...
cd proxy && docker-compose --env-file ../.env down -v || echo ""
cd dmas && docker-compose --env-file ../.env down || echo ""
cd monitoring && docker-compose --env-file ../.env down -v || echo ""
docker volume rm dmas-long-context-memory_locomo-data 2>/dev/null || echo ""
dmas-long-context-memory_locomo-data
docker volume rm dmas-long-context-memory_qdrant-data 2>/dev/null || echo ""
dmas-long-context-memory_qdrant-data
docker volume rm dmas-long-context-memory_neo4j-data 2>/dev/null || echo ""
dmas-long-context-memory_neo4j-data
docker volume rm dmas-long-context-memory_neo4j-logs 2>/dev/null || echo ""
dmas-long-context-memory_neo4j-logs
docker volume rm prometheus-data 2>/dev/null || echo ""

docker network rm dmas-network 2>/dev/null || echo ""

Reset complete! Ollama volume preserved.
make: Leaving directory '/home/jacobbista/Documents/RM&

 Volume monitoring_prometheus-data  Removing
 Volume monitoring_prometheus-data  Removed


### Environment Variables


In [146]:
import os
import re

def set_env_var(var_name: str, new_value: str, env_file: str = '../.env') -> None:
    env_file = os.path.abspath(env_file)
    content = ''
    if os.path.exists(env_file):
        with open(env_file, 'r') as f:
            content = f.read()
    pattern = re.compile(rf'^{re.escape(var_name)}=.*$', re.MULTILINE)
    if pattern.search(content):
        content = pattern.sub(f'{var_name}={new_value}', content)
    else:
        if content and not content.endswith('\n'):
            content += '\n'
        content += f'{var_name}={new_value}\n'
    with open(env_file, 'w') as f:
        f.write(content)

set_env_var('MEMORY_BACKEND', MEMORY_BACKEND)
set_env_var('TOXIC_LATENCY', str(TOXIC_LATENCY))
set_env_var('TOXIC_BANDWIDTH', str(TOXIC_BANDWIDTH))
set_env_var('TOXIC_SLOW_CLOSE', str(TOXIC_SLOW_CLOSE))
set_env_var('TOXIC_TIMEOUT', str(TOXIC_TIMEOUT))
set_env_var('TOXIC_SLICER', str(TOXIC_SLICER))
set_env_var('TOXIC_LIMIT_DATA', str(TOXIC_LIMIT_DATA))
set_env_var('TOXIC_RESET_PEER', str(TOXIC_RESET_PEER))

### Build


#### Monitoring


In [5]:
!make -C ../ build-monitoring

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building monitoring services...
cd monitoring && docker-compose --env-file ../.env build --no-cache --pull
Monitoring build complete!
make: Leaving directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'


#### Proxy


In [6]:
!make -C ../ build-proxy

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building proxy services...
cd proxy && docker-compose --env-file ../.env build --no-cache --pull
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0/0
 ⠋ Service openai-proxy  Building                                          0.1s 
 ⠋ Service ollama-proxy  Building                                          0.1s 
[+] Building 0/2
 ⠙ Service openai-proxy  Building                                          0.2s 
 ⠙ Service ollama-proxy  Building                                          0.2s 
[+] Building 0.2s (2/3)                                          docker:default
 => [openai-proxy internal] load build definition from Dockerfile          0.0s
 => => transferring dockerfile: 268B                                       0.0s
 => [ollama-proxy internal] load build definition from Dockerfile          0.0s
 => => transferring dockerfile: 268B             

#### Dmas


In [7]:
!make -C ../ build-dmas

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Building dmas services...
cd dmas && docker-compose --env-file ../.env build --no-cache --pull
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0/0
 ⠋ Service locomo     Building                                             0.1s 
 ⠋ Service memory     Building                                             0.1s 
 ⠋ Service responder  Building                                             0.1s 
[+] Building 0.2s (3/4)                                          docker:default
 => [locomo internal] load build definition from Dockerfile                0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [responder internal] load build definition from Dockerfile             0.0s
 => => transferring dockerfile: 279B                                       0.0s
 => [memory internal] load build definition from Dockerfile          

### System Up


In [99]:
!make -C ../ up

make: Entering directory '/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory'
Setting DOCKER_GID=
Starting services on ...
Docker GID:
MEMORY_BACKEND: MEMORY_BACKEND=graphiti
docker network rm dmas-network 2>/dev/null || echo ""

docker network create dmas-network 2>/dev/null || echo "Network exists"
e397923d45e23fc882fd9855b731cae12399cfb9eb52e0572fc5e218dc60e3c5
cd proxy && docker-compose --env-file ../.env up -d --force-recreate --remove-orphans


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[+] Running 0/0
 ⠋ Container openai-proxy  Starting                                        0.1s 
 ⠋ Container ollama-proxy  Starting                                        0.1s 
[+] Running 2/2
 ✔ Container openai-proxy  Started                                         0.1s 
 ✔ Container ollama-proxy  Started                                         0.1s 
cd dmas && docker-compose --env-file ../.env up -d --force-recreate --remove-orphans
[+] Running 3/0
 ✔ Network dmas-long-context-memory_memory-internal  Created               0.0s 
 ⠋ Container toxiproxy                               Starting              0.1s 
 ⠋ Container responder                               Starting              0.1s 
 ⠋ Container ollama                                  Starting              0.1s 
 ⠋ Container locomo                                  Starting              0.1s 
 ⠋ Container neo4j                                   Starting              0.1s 
 ⠋ Container qdrant                                  Star

### Memory Restart


In [162]:
!cd ../dmas && export $(grep MEMORY_BACKEND ../.env) && docker-compose --env-file ../.env up -d --force-recreate --no-deps memory

[+] Running 0/0
 ⠋ Container memory  Recreate                                              0.1s 
[+] Running 0/1
 ⠙ Container memory  Recreate                                              0.2s 
[+] Running 0/1
 ⠹ Container memory  Recreate                                              0.3s 
[+] Running 0/1
 ⠸ Container memory  Recreate                                              0.4s 
[+] Running 0/1
 ⠼ Container memory  Recreate                                              0.5s 
[+] Running 0/1
 ⠴ Container memory  Recreate                                              0.6s 
[+] Running 0/1
 ⠦ Container memory  Starting                                              0.7s 
[+] Running 1/1
 ✔ Container memory  Started                                               0.7s 


In [157]:
!docker exec memory printenv MEMORY_BACKEND

graphiti


### Toxiproxy Setup


#### Profiles


In [158]:
PROFILE_UNCONSTRAINED = {
    "TOXIC_LATENCY": 0,
    "TOXIC_BANDWIDTH": 0,
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 0,
}

PROFILE_5G_REALISTIC = {
    "TOXIC_LATENCY": 200,      # ms
    "TOXIC_BANDWIDTH": 1000,   # KB/s ≈ 8 Mb/s
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 2,     # 2% reset
}


In [159]:
import requests

TOXIPROXY_API = "http://localhost:8474"

def ensure_proxy(name, listen, upstream):
    resp = requests.get(f"{TOXIPROXY_API}/proxies/{name}")
    if resp.status_code == 200:
        return resp.json()
    resp = requests.post(f"{TOXIPROXY_API}/proxies", json={
        "name": name,
        "listen": listen,
        "upstream": upstream,
    })
    resp.raise_for_status()
    return resp.json()

def clear_toxics(proxy_name):
    resp = requests.get(f"{TOXIPROXY_API}/proxies/{proxy_name}/toxics")
    if resp.status_code != 200:
        return
    for toxic in resp.json():
        requests.delete(f"{TOXIPROXY_API}/proxies/{proxy_name}/toxics/{toxic['name']}")

def base_setup():
    # Create the two proxies if they don't already exist
    ensure_proxy("memory-proxy",    "0.0.0.0:18005", "memory:8002")
    ensure_proxy("responder-proxy", "0.0.0.0:18006", "responder:8003")

def apply_network_profile(profile: dict):
    global TOXIC_LATENCY, TOXIC_BANDWIDTH, TOXIC_SLOW_CLOSE
    global TOXIC_TIMEOUT, TOXIC_SLICER, TOXIC_LIMIT_DATA, TOXIC_RESET_PEER
    
    # Update Python variables
    TOXIC_LATENCY     = int(profile["TOXIC_LATENCY"])
    TOXIC_BANDWIDTH   = int(profile["TOXIC_BANDWIDTH"])
    TOXIC_SLOW_CLOSE  = int(profile["TOXIC_SLOW_CLOSE"])
    TOXIC_TIMEOUT     = int(profile["TOXIC_TIMEOUT"])
    TOXIC_SLICER      = int(profile["TOXIC_SLICER"])
    TOXIC_LIMIT_DATA  = int(profile["TOXIC_LIMIT_DATA"])
    TOXIC_RESET_PEER  = int(profile["TOXIC_RESET_PEER"])

    print(f"Applying profile: {profile}")

    # Update the .env file
    set_env_var("TOXIC_LATENCY", str(TOXIC_LATENCY))
    set_env_var("TOXIC_BANDWIDTH", str(TOXIC_BANDWIDTH))
    set_env_var("TOXIC_SLOW_CLOSE", str(TOXIC_SLOW_CLOSE))
    set_env_var("TOXIC_TIMEOUT", str(TOXIC_TIMEOUT))
    set_env_var("TOXIC_SLICER", str(TOXIC_SLICER))
    set_env_var("TOXIC_LIMIT_DATA", str(TOXIC_LIMIT_DATA))
    set_env_var("TOXIC_RESET_PEER", str(TOXIC_RESET_PEER))

    # Configure the Toxiproxy proxies
    base_setup()
    for name in ["memory-proxy", "responder-proxy"]:
        clear_toxics(name)

        if TOXIC_LATENCY > 0:
            resp = requests.post(f"{TOXIPROXY_API}/proxies/{name}/toxics", json={
                "name": "latency",
                "type": "latency",
                "stream": "downstream",
                "attributes": {
                    "latency": TOXIC_LATENCY,
                    "jitter": TOXIC_LATENCY // 4,
                }
            })
            resp.raise_for_status()
            print(f"  {name}: added latency toxic ({TOXIC_LATENCY}ms)")

        if TOXIC_BANDWIDTH > 0:
            resp = requests.post(f"{TOXIPROXY_API}/proxies/{name}/toxics", json={
                "name": "bandwidth",
                "type": "bandwidth",
                "stream": "downstream",
                "attributes": {
                    "rate": TOXIC_BANDWIDTH * 1024,
                }
            })
            resp.raise_for_status()
            print(f"  {name}: added bandwidth toxic ({TOXIC_BANDWIDTH}KB/s)")

        if TOXIC_RESET_PEER > 0:
            resp = requests.post(f"{TOXIPROXY_API}/proxies/{name}/toxics", json={
                "name": "reset_peer",
                "type": "reset_peer",
                "stream": "downstream",
                "toxicity": TOXIC_RESET_PEER / 100.0,
                "attributes": {}
            })
            resp.raise_for_status()
            print(f"  {name}: added reset_peer toxic ({TOXIC_RESET_PEER}%)")
    
    print("Profile applied successfully")


#### Apply the profile


In [160]:
# apply_network_profile(PROFILE_5G_REALISTIC)
apply_network_profile(PROFILE_UNCONSTRAINED)

Applying profile: {'TOXIC_LATENCY': 0, 'TOXIC_BANDWIDTH': 0, 'TOXIC_SLOW_CLOSE': 0, 'TOXIC_TIMEOUT': 0, 'TOXIC_SLICER': 0, 'TOXIC_LIMIT_DATA': 0, 'TOXIC_RESET_PEER': 0}
Profile applied successfully


In [152]:
from time import sleep
import requests
import json

sleep(5)

response = requests.get("http://localhost:8474/proxies")
print(json.dumps(response.json(), indent=2))

{
  "memory-proxy": {
    "name": "memory-proxy",
    "listen": "[::]:18005",
    "upstream": "memory:8002",
    "enabled": true,
    "Logger": {},
    "toxics": []
  },
  "responder-proxy": {
    "name": "responder-proxy",
    "listen": "[::]:18006",
    "upstream": "responder:8003",
    "enabled": true,
    "Logger": {},
    "toxics": []
  }
}


## Populate Memory


### Conversation index and session


In [153]:
conversation_index = 0
sessions = 19 # 1-19

In [154]:
import time
import json
import requests

PROM_URL = "http://localhost:9090/api/v1/query_range"
PROM_INSTANT_URL = "http://localhost:9090/api/v1/query"

def query_range(query, start, end, step="1s"):
    params = {"query": query, "start": start, "end": end, "step": step}
    r = requests.get(PROM_URL, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    total = 0.0
    for result in data['data']['result']:
        for v in result['values']:
            total += float(v[1])
    return total

def get_instant_tokens(prometheus_url=PROM_INSTANT_URL):
    query = 'sum(openai_tokens_total{type="total"})'
    r = requests.get(prometheus_url, params={"query": query}, timeout=10)
    if r.status_code == 200:
        data = r.json()
        if data['status'] == 'success' and data['data']['result']:
            return float(data['data']['result'][0]['value'][1])
    return 0

def normalize_model_name(model_name):
    import re
    normalized = re.sub(r'-\d{4}-\d{2}-\d{2}$', '', model_name)
    return normalized

def get_openai_cost(start, end, pricing_file="openai_prices.json", tier="standard"):
    try:
        with open(pricing_file, 'r') as f:
            pricing_data = json.load(f)
        
        pricing_list = pricing_data['pricing']['text_tokens'][tier]
        text_pricing = {item['model']: {'input': item['input'], 'output': item['output']} for item in pricing_list}
        
        embedding_pricing = {item['model']: item['cost'] for item in pricing_data['pricing']['embeddings']}
        
        total_cost = 0.0
        
        for token_type in ['prompt', 'completion']:
            query = f'sum by (model) (openai_tokens_total{{type="{token_type}",endpoint="chat.completions"}})'
            
            r_before = requests.get(PROM_INSTANT_URL, params={"query": query, "time": start}, timeout=10)
            r_after = requests.get(PROM_INSTANT_URL, params={"query": query, "time": end + 2}, timeout=10)
            
            if r_before.status_code == 200 and r_after.status_code == 200:
                data_before = r_before.json()
                data_after = r_after.json()
                
                before_values = {}
                if data_before['status'] == 'success':
                    for result in data_before['data']['result']:
                        model = result['metric'].get('model', 'unknown')
                        before_values[model] = float(result['value'][1])
                
                if data_after['status'] == 'success':
                    for result in data_after['data']['result']:
                        model = result['metric'].get('model', 'unknown')
                        after_value = float(result['value'][1])
                        before_value = before_values.get(model, 0)
                        tokens = after_value - before_value
                        
                        if tokens > 0:
                            normalized_model = normalize_model_name(model)
                            
                            if model in text_pricing:
                                pricing_model = model
                            elif normalized_model in text_pricing:
                                pricing_model = normalized_model
                            else:
                                continue
                            
                            price_per_million = text_pricing[pricing_model]['input'] if token_type == 'prompt' else text_pricing[pricing_model]['output']
                            if price_per_million:
                                cost = (tokens / 1_000_000) * price_per_million
                                total_cost += cost
        
        query = f'sum by (model) (openai_tokens_total{{endpoint="embeddings"}})'
        
        r_before = requests.get(PROM_INSTANT_URL, params={"query": query, "time": start}, timeout=10)
        r_after = requests.get(PROM_INSTANT_URL, params={"query": query, "time": end + 2}, timeout=10)
        
        if r_before.status_code == 200 and r_after.status_code == 200:
            data_before = r_before.json()
            data_after = r_after.json()
            
            before_values = {}
            if data_before['status'] == 'success':
                for result in data_before['data']['result']:
                    model = result['metric'].get('model', 'unknown')
                    before_values[model] = float(result['value'][1])
            
            if data_after['status'] == 'success':
                for result in data_after['data']['result']:
                    model = result['metric'].get('model', 'unknown')
                    after_value = float(result['value'][1])
                    before_value = before_values.get(model, 0)
                    tokens = after_value - before_value
                    
                    if tokens > 0 and model in embedding_pricing:
                        cost = (tokens / 1_000_000) * embedding_pricing[model]
                        total_cost += cost
        
        return total_cost
    except Exception as e:
        print(f"Error calculating OpenAI cost: {e}")
        import traceback
        traceback.print_exc()
        return 0

def measure_function(func, *args, min_duration=15, **kwargs):
    tokens_before = get_instant_tokens()

    start = time.time()
    print(f"Starting function '{func.__name__}' at {start}")
    result = func(*args, **kwargs)
    end = time.time()
    print(f"Function '{func.__name__}' completed at {end}")
    actual_duration = end - start

    duration_s = max(int(actual_duration), min_duration)
    if actual_duration < min_duration:
        time.sleep(min_duration - actual_duration)
        end = time.time()

    time.sleep(2)
    
    tokens_after = get_instant_tokens()

    cpu_edge = query_range(f'sum(increase(docker_container_cpu_usage_total{{group="edge"}}[{duration_s}s]))', start, end)
    cpu_cloud = query_range(f'sum(increase(docker_container_cpu_usage_total{{group="cloud"}}[{duration_s}s]))', start, end)
    ram_edge = query_range(f'avg_over_time(docker_container_mem_usage{{group="edge"}}[{duration_s}s])', start, end)
    ram_cloud = query_range(f'avg_over_time(docker_container_mem_usage{{group="cloud"}}[{duration_s}s])', start, end)
    
    disk_edge_read = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_read{{group="edge"}}[{duration_s}s]))', start, end)
    disk_edge_write = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_write{{group="edge"}}[{duration_s}s]))', start, end)
    disk_edge = disk_edge_read + disk_edge_write
    
    disk_cloud_read = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_read{{group="cloud"}}[{duration_s}s]))', start, end)
    disk_cloud_write = query_range(f'sum(increase(docker_container_blkio_io_service_bytes_recursive_write{{group="cloud"}}[{duration_s}s]))', start, end)
    disk_cloud = disk_cloud_read + disk_cloud_write
    
    network_edge = query_range(f'sum(increase(docker_container_net_rx_bytes{{group="edge"}}[{duration_s}s]))', start, end)
    network_cloud = query_range(f'sum(increase(docker_container_net_rx_bytes{{group="cloud"}}[{duration_s}s]))', start, end)
    
    tokens = int(tokens_after - tokens_before)
    
    token_cost = get_openai_cost(start, end)

    metrics = {
        "cpu_edge_ns": cpu_edge,
        "cpu_cloud_ns": cpu_cloud,
        "ram_edge_bytes": ram_edge,
        "ram_cloud_bytes": ram_cloud,
        "disk_edge_bytes": disk_edge,
        "disk_cloud_bytes": disk_cloud,
        "network_edge_bytes": network_edge,
        "network_cloud_bytes": network_cloud,
        "openai_tokens": tokens,
        "openai_cost_usd": token_cost,
        "api_latency_s": actual_duration,
        "metric_window_s": duration_s
    }

    return {"result": result, "metrics": metrics}

Optional: run `docker logs memory -f`


### Load conversations


In [155]:
import shutil
import pandas as pd

def load_session(session_id):
    requests.post(f"http://localhost:8003/conversation/load/{conversation_index}/session/{session_id}")
    
memories = []

for i in range(1, sessions):
    print(f"Loading session: {i}")
    output = measure_function(load_session, i)
    row = {
        "memory": MEMORY_BACKEND,
        "time": time.time(),
        "conversation_index": conversation_index,
        "session_id": i,
        **output["metrics"],
        "toxic_latency": TOXIC_LATENCY,
        "toxic_bandwidth": TOXIC_BANDWIDTH,
        "toxic_slow_close": TOXIC_SLOW_CLOSE,
        "toxic_timeout": TOXIC_TIMEOUT,
        "toxic_slicer": TOXIC_SLICER,
        "toxic_limit_data": TOXIC_LIMIT_DATA,
        "toxic_reset_peer": TOXIC_RESET_PEER
    }
    print(row)
    memories.append(row)

Loading session: 1
Starting function 'load_session' at 1763227169.841472
Function 'load_session' completed at 1763227507.1098576
{'memory': 'graphiti', 'time': 1763227509.142658, 'conversation_index': 0, 'session_id': 1, 'cpu_edge_ns': 845516093833.3342, 'cpu_cloud_ns': 9303848300458.338, 'ram_edge_bytes': 32400073153.70919, 'ram_cloud_bytes': 358010007527.6968, 'disk_edge_bytes': 0.0, 'disk_cloud_bytes': 790990994.2857127, 'network_edge_bytes': 36827847.44642846, 'network_cloud_bytes': 2533476873.426262, 'openai_tokens': 137410, 'openai_cost_usd': 7.432e-05, 'api_latency_s': 337.2683856487274, 'metric_window_s': 337, 'toxic_latency': 0, 'toxic_bandwidth': 0, 'toxic_slow_close': 0, 'toxic_timeout': 0, 'toxic_slicer': 0, 'toxic_limit_data': 0, 'toxic_reset_peer': 0}
Loading session: 2
Starting function 'load_session' at 1763227509.144392
Function 'load_session' completed at 1763227873.0890703
{'memory': 'graphiti', 'time': 1763227875.120882, 'conversation_index': 0, 'session_id': 2, 'cp

KeyboardInterrupt: 

In [161]:
if memories:
    df = pd.DataFrame(memories)
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    is_constrained = (
        int(TOXIC_LATENCY) > 0 or
        int(TOXIC_BANDWIDTH) > 0 or
        int(TOXIC_SLOW_CLOSE) > 0 or
        int(TOXIC_TIMEOUT) > 0 or
        int(TOXIC_SLICER) > 0 or
        int(TOXIC_LIMIT_DATA) > 0 or
        int(TOXIC_RESET_PEER) > 0
    )
    
    if is_constrained:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_constrained.csv"
    else:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions.csv"
    
    output_path = os.path.join(output_dir, output_file)
    
    if os.path.exists(output_path):
        if is_constrained:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_constrained_backup.csv")
        else:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_sessions_backup.csv")
        shutil.move(output_path, backup_path)
        print(f"Existing file backed up to {backup_path}")
    
    df.to_csv(output_path, index=False)
    print(f"Saved new results to {output_path}")

Existing file backed up to results/graphiti_conv_0_sessions_backup.csv
Saved new results to results/graphiti_conv_0_sessions.csv


### Q&A experiment


In [65]:
import requests

questions = requests.get(f"http://localhost:8002/conversations/index/{conversation_index}/questions").json()
print(questions)

{'conversation_index': 0, 'sample_id': 'conv-26', 'total_questions': 199, 'questions': [{'sample_id': 'conv-26', 'question_id': 'conv-26_q_0', 'question': 'When did Caroline go to the LGBTQ support group?', 'answer': '7 May 2023', 'category': 2, 'evidence': ['D1:3']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_1', 'question': 'When did Melanie paint a sunrise?', 'answer': '2022', 'category': 2, 'evidence': ['D1:12']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_2', 'question': 'What fields would Caroline be likely to pursue in her educaton?', 'answer': 'Psychology, counseling certification', 'category': 3, 'evidence': ['D1:9', 'D1:11']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_3', 'question': 'What did Caroline research?', 'answer': 'Adoption agencies', 'category': 1, 'evidence': ['D2:8']}, {'sample_id': 'conv-26', 'question_id': 'conv-26_q_4', 'question': "What is Caroline's identity?", 'answer': 'Transgender woman', 'category': 1, 'evidence': ['D1:5']}, {'sa

In [66]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [67]:
from difflib import SequenceMatcher
from sentence_transformers import SentenceTransformer, util

_model = SentenceTransformer('all-MiniLM-L6-v2')

def check_similarity_string(text1: str, text2: str) -> float:
    if not text1 or not text2:
        return 0.0
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()


def check_similarity_semantic(text1: str, text2: str) -> float:
    if not text1 or not text2:
        return 0.0
    emb1 = _model.encode(text1, convert_to_tensor=True)
    emb2 = _model.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2).item())

/home/jacobbista/Documents/RM&SW/Experiment/dmas-long-context-memory/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
import pandas as pd
import time
import requests

def ask(question):
    resp = requests.post("http://localhost:8003/ask", params={"question": question})
    return resp.json().get("answer")

qa = []

total_questions = len(questions.get("questions", []))

for idx, question in enumerate(questions.get("questions", []), start=1):
    print(f"[{idx}/{total_questions}] Asking question: {question.get('question')}")
    output = measure_function(ask, question.get("question"))
    print(f"[{idx}/{total_questions}] Answer:", output["result"])
    
    answer_received = output["result"]
    metrics = output["metrics"]
    
    similarity_string = check_similarity_string(question.get("answer"), answer_received)
    similarity_semantic = check_similarity_semantic(question.get("answer"), answer_received)
    
    row = {
        "memory": MEMORY_BACKEND,
        "time": time.time(),
        "conversation_index": conversation_index,
        "question": question.get("question"),
        "answer_actual": question.get("answer"),
        "answer_received": answer_received,
        "similarity_string": similarity_string,
        "similarity_semantic": similarity_semantic,
        "similarity": (similarity_string + similarity_semantic) / 2,
        **metrics,
        "toxic_latency": TOXIC_LATENCY,
        "toxic_bandwidth": TOXIC_BANDWIDTH,
        "toxic_slow_close": TOXIC_SLOW_CLOSE,
        "toxic_timeout": TOXIC_TIMEOUT,
        "toxic_slicer": TOXIC_SLICER,
        "toxic_limit_data": TOXIC_LIMIT_DATA,
        "toxic_reset_peer": TOXIC_RESET_PEER
    }
    
    print(row)
    qa.append(row)

[1/199] Asking question: When did Caroline go to the LGBTQ support group?
Starting function 'ask' at 1763218631.855764
Function 'ask' completed at 1763218645.6569924
[1/199] Answer: Caroline went to an LGBTQ support group on July 16, 2023.
{'memory': 'mem0', 'time': 1763218649.114257, 'conversation_index': 0, 'question': 'When did Caroline go to the LGBTQ support group?', 'answer_actual': '7 May 2023', 'answer_received': 'Caroline went to an LGBTQ support group on July 16, 2023.', 'similarity_string': 0.23880597014925373, 'similarity_semantic': 0.37025171518325806, 'similarity': 0.3045288426662559, 'cpu_edge_ns': 83853050566.92825, 'cpu_cloud_ns': 6967956416.478728, 'ram_edge_bytes': 2042050969.6000001, 'ram_cloud_bytes': 14782508236.799997, 'disk_edge_bytes': 4009556224.877714, 'disk_cloud_bytes': 35108.57142857143, 'network_edge_bytes': 38869.50736277081, 'network_cloud_bytes': 154352.29439857948, 'openai_tokens': 2348, 'openai_cost_usd': 0.005849249999999999, 'api_latency_s': 13.801

In [69]:
if qa:
    df = pd.DataFrame(qa)
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    is_constrained = (
        int(TOXIC_LATENCY) > 0 or
        int(TOXIC_BANDWIDTH) > 0 or
        int(TOXIC_SLOW_CLOSE) > 0 or
        int(TOXIC_TIMEOUT) > 0 or
        int(TOXIC_SLICER) > 0 or
        int(TOXIC_LIMIT_DATA) > 0 or
        int(TOXIC_RESET_PEER) > 0
    )
    
    if is_constrained:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_constrained.csv"
    else:
        output_file = f"{MEMORY_BACKEND}_conv_{conversation_index}_qa.csv"
    
    output_path = os.path.join(output_dir, output_file)
    
    if os.path.exists(output_path):
        backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_backup.csv")
        if is_constrained:
            backup_path = os.path.join(output_dir, f"{MEMORY_BACKEND}_conv_{conversation_index}_qa_constrained_backup.csv")
        shutil.move(output_path, backup_path)
        print(f"Existing file backed up to {backup_path}")
    
    df.to_csv(output_path, index=False)
    print(f"Saved new results to {output_path}")

Existing file backed up to results/mem0_conv_0_qa_backup.csv
Saved new results to results/mem0_conv_0_qa.csv


In [70]:
if qa:
    df_old = pd.read_csv('results/mem0_conv_0_qa_backup.csv')
    df_new = pd.read_csv('results/mem0_conv_0_qa.csv')
    
    metrics = ['similarity', 'similarity_string', 'similarity_semantic']
    
    print("=" * 60)
    print("QA COMPARISON: BACKUP vs CURRENT")
    print("=" * 60)
    
    for metric in metrics:
        old_mean = df_old[metric].mean()
        new_mean = df_new[metric].mean()
        diff = new_mean - old_mean
        pct_change = (diff / old_mean * 100) if old_mean != 0 else 0
        
        print(f"\n{metric.upper()}:")
        print(f"  Backup:  {old_mean:.4f}")
        print(f"  Current: {new_mean:.4f}")
        print(f"  Change:  {diff:+.4f} ({pct_change:+.2f}%)")
    
    improved = (df_new['similarity'] > df_old['similarity']).sum()
    degraded = (df_new['similarity'] < df_old['similarity']).sum()
    unchanged = (df_new['similarity'] == df_old['similarity']).sum()
    
    print(f"\n{'=' * 60}")
    print(f"QUESTIONS IMPROVED:  {improved}/{len(df_new)} ({improved/len(df_new)*100:.1f}%)")
    print(f"QUESTIONS DEGRADED:  {degraded}/{len(df_new)} ({degraded/len(df_new)*100:.1f}%)")
    print(f"QUESTIONS UNCHANGED: {unchanged}/{len(df_new)} ({unchanged/len(df_new)*100:.1f}%)")
    print(f"{'=' * 60}")

QA COMPARISON: BACKUP vs CURRENT

SIMILARITY:
  Backup:  0.2293
  Current: 0.2268
  Change:  -0.0025 (-1.10%)

SIMILARITY_STRING:
  Backup:  0.2229
  Current: 0.2224
  Change:  -0.0004 (-0.18%)

SIMILARITY_SEMANTIC:
  Backup:  0.2358
  Current: 0.2311
  Change:  -0.0046 (-1.96%)

QUESTIONS IMPROVED:  46/199 (23.1%)
QUESTIONS DEGRADED:  45/199 (22.6%)
QUESTIONS UNCHANGED: 108/199 (54.3%)
